In [75]:
import os
import sys
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings
import tensorflow as tf
import pickle

warnings.filterwarnings('ignore')
%matplotlib inline

In [76]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [77]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [78]:
## Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [79]:
### Encode categorical variable
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [80]:
# One-hot encode 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [81]:
### Combine one-hot encoded columns with original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [82]:
### Split the data into features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [83]:
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0


In [84]:
### Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Scaling these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [85]:
# Save the encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Regression Problem statement

In [86]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [88]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from sklearn.base import BaseEstimator, RegressorMixin
# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np

# class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
#     def __init__(self, neurons=64, activation='relu', optimizer='adam', batch_size=32, epochs=100):
#         self.neurons = neurons
#         self.activation = activation
#         self.optimizer = optimizer
#         self.batch_size = batch_size
#         self.epochs = epochs
#         self.model = self.build_model()

#     def build_model(self):
#         model = Sequential([
#             Dense(self.neurons, activation=self.activation, input_shape=(X_train.shape[1],)),  
#             Dense(self.neurons // 2, activation=self.activation),  
#             Dense(1)  # Output layer
#         ])
#         model.compile(optimizer=self.optimizer, loss='mean_absolute_error', metrics=['mae'])
#         return model

#     def fit(self, X, y):
#         self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=0)
#         return self

#     def predict(self, X):
#         return self.model.predict(X)

# # Define the parameter distribution
# param_dist = {
#     'neurons': [32, 64, 128],
#     'activation': ['relu', 'tanh'],
#     'optimizer': ['adam', 'rmsprop', 'sgd'],
#     'batch_size': [16, 32, 64],
#     'epochs': [50, 100, 200]
# }

# # Create the RandomizedSearchCV object
# random_search = RandomizedSearchCV(
#     estimator=KerasRegressorWrapper(),
#     param_distributions=param_dist,
#     n_iter=10,      # Try 10 different combinations
#     cv=3,           # 3-Fold Cross Validation
#     verbose=2,
#     random_state=42
# )

# # Fit the model
# random_search.fit(X_train, y_train)
# print("Best parameters found:", random_search.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[CV] END activation=tanh, batch_size=64, epochs=200, neurons=64, optimizer=sgd; total time=  31.4s
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[CV] END activation=tanh, batch_size=64, epochs=200, neurons=64, optimizer=sgd; total time=  29.4s
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[CV] END activation=tanh, batch_size=64, epochs=200, neurons=64, optimizer=sgd; total time=  29.2s
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[CV] END activation=tanh, batch_size=32, epochs=50, neurons=32, optimizer=rmsprop; total time=  12.9s
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[CV] END activation=tanh, batch_size=32, epochs=50, neurons=32, optimizer=rmsprop; total time=  12.9s
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[CV] END activation=tanh, batch_size=32, epochs=50, neurons=32, optimizer=rmsprop; total time=  13.1s
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[CV] END activation=tanh, batch_size=32, epochs=2

KeyboardInterrupt: 

In [89]:
### Build Our ANN models
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  ### Hidden Layer 1 Connected with input Layer
        Dense(32, activation='relu'),                                   ### Hidden Layer 2
        Dense(1)                                                       ### Output Layer for Regression
    ]
)

### Compile the Models
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

Model: "sequential_47"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_141 (Dense)               │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_142 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_143 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [90]:
### Setup the TensorBoard
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tf_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [91]:
### Setup the EarlyStopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [92]:
### Train the model
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=100,
    callbacks = [tf_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 101019.0234 - mae: 101019.0234 - val_loss: 98513.0234 - val_mae: 98513.0234
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 99137.8906 - mae: 99137.8906 - val_loss: 96961.6172 - val_mae: 96961.6172
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 97010.5156 - mae: 97010.5156 - val_loss: 92998.2344 - val_mae: 92998.2344
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 94011.3906 - mae: 94011.3906 - val_loss: 86403.9766 - val_mae: 86403.9766
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 85399.7266 - mae: 85399.7266 - val_loss: 78014.8516 - val_mae: 78014.8516
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 77932.8984 - mae: 77932.8984 - val_loss: 69233.1328 - val_mae: 69233.1328
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 68748.2500 - mae: 68748.2500 - val_loss: 61523.7695 - val_mae: 61523.7695
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [93]:
%load_ext tensorboard

In [96]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 20916), started 0:00:30 ago. (Use '!kill 20916' to kill it.)

In [97]:
### Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51098.4570 - mae: 51098.4570
Test MAE: 50226.78125


In [99]:
model.save('regression_model.h5')